In [1]:
import pandas as pd
import pathlib
import w_analyse_netflix as w_a
debug, yaml_data = w_a.ProcessYAML('netflix.yaml') 

file = yaml_data['view']
path = pathlib.Path.cwd() / file
df_view = pd.read_csv(path)
df_view.shape

file = yaml_data['devices']
path = pathlib.Path.cwd() / file
df_devices = pd.read_csv(path)
df_devices.shape


(37, 12)

In [4]:
#df_devices.head(1)
list(df_devices['Esn'].unique())

['NFCDIE-03-HNFTN4HDFYFKPQCTCKXQ17RAU0KQ1P',
 'SONY-CTV140000000000000007457208',
 'NFAPPL-01-IPAD7=5-1D7FD2E25DC8D08A8581A7A410AEB866A9B27842F2FB756F918B160C8B102B0D',
 'NFANDROID2-PRV-HIMEDIADISHSV10-HIMEDSV10-10094-EFF3AEBDC7A55C4B6C673B77E255D7E241EC48FD787B43F3F3FB25F6041754DE',
 'NFAPPL-01-IPAD4=2-DE4126F171ACB5D089ACFA06509D88809E69D64D56E3B4A0198D594D93FEBB66',
 'NFCDIE-03-HG1FETYD0XCMTNR5XY4ARM6KW65DT9',
 'NFCDCH-02-FZQGFH54CF5YWAMEJL28HHQ41XDR6H',
 'NFCDCH-02-FUFW4JH1XV3F1PEYC8U201T1LKDUAN',
 'GOOGEAR0010000000000000021919795',
 'NFCDCH-01-FFLW2CT4M85Y4E0CF2551MTFJMMJ1C',
 'NFCDCH-AP-GZ30Y2TT814R3NEDTUWUAJJ18QT5LE',
 'NFCDCH-01-GPDU57H2XL7A2NC43E9EQH8U3AQ490',
 'NFANDROID1-PRV-T-L3-ROCKCH96=MAX=-4445-0E43AF9CBA596398A9E1B8B83E6DFA3D275B26CD9D783DD860895B9193799B5D1',
 'NFCDCH-02-GPXGRNX466Y50MGR847R7E5AY7X8YE',
 'GOOGNEW0010000000000000000308216',
 'NFANDROID1-PRV-T-L3-HENA=T10HR2S-4445-081C03E1B2B0380BAE3C58BD3C0E0D2354E8446CEE213ABF29C6ECC985C02C9FF',
 'NFCDCH-01-G00G5WWPC0

In [ ]:
df_view2 = w_a.TotalTimePerProfile(df_view)
print (df_view2)

In [ ]:
plist = w_a.ProfileList(df_view)
print(plist)
for name in plist:
    print (name)


In [ ]:
#dropping some columns
df_view = df_view.drop(['Profile Name', 'Attributes', 'Supplemental Video Type', 'Device Type', 'Bookmark', 'Latest Bookmark', 'Country'], axis=1)
df_view.head(1)

In [ ]:
df_view.dtypes

In [ ]:
df_view['Start Time'] = pd.to_datetime(df_view['Start Time'], utc=True)
df_view.dtypes

In [ ]:
#Set this to your local TimeZone if desired
import pytz
pytz.all_timezones

In [ ]:
# change the Start Time column into the dataframe's index
df_view = df_view.set_index('Start Time')

# convert from UTC timezone to eastern time
df_view.index = df_view.index.tz_convert('Pacific/Auckland')

# reset the index so that Start Time becomes a column again
df_view = df_view.reset_index()

#double-check that it worked
df_view.head(1)

In [ ]:
df_view['Duration'] = pd.to_timedelta(df_view['Duration'])
df_view.dtypes

In [ ]:
shameless = df_view[df_view['Title'].str.contains('Shameless', regex=False)]
shameless.shape

In [ ]:
shameless = shameless[(shameless['Duration'] > '0 days 00:01:00')]
shameless.shape

In [ ]:
shameless['Duration'].sum()

In [ ]:
shameless

In [ ]:
shameless['weekday'] = shameless['Start Time'].dt.weekday
shameless['hour'] = shameless['Start Time'].dt.hour
shameless.head(1)   

In [ ]:
%matplotlib inline
import matplotlib

# set our categorical and define the order so the days are plotted Monday-Sunday
shameless['weekday'] = pd.Categorical(shameless['weekday'], categories=
    [0,1,2,3,4,5,6],
    ordered=True)

# create shameless_by_day and count the rows for each weekday, assigning the result to that variable
shameless_by_day = shameless['weekday'].value_counts()

# sort the index using our categorical, so that Monday (0) is first, Tuesday (1) is second, etc.
shameless_by_day = shameless_by_day.sort_index()

# optional: update the font size to make it a bit larger and easier to read
matplotlib.rcParams.update({'font.size': 22})

# plot shameless_by_day as a bar chart with the listed size and title
shameless_by_day.plot(kind='bar', figsize=(20,10), title='Shameless Episodes Watched by Day')

In [ ]:
# set our categorical and define the order so the hours are plotted 0-23
shameless['hour'] = pd.Categorical(shameless['hour'], categories=
    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
    ordered=True)

# create shameless_by_hour and count the rows for each hour, assigning the result to that variable
shameless_by_hour = shameless['hour'].value_counts()

# sort the index using our categorical, so that midnight (0) is first, 1 a.m. (1) is second, etc.
shameless_by_hour = shameless_by_hour.sort_index()

# plot shameless_by_hour as a bar chart with the listed size and title
shameless_by_hour.plot(kind='bar', figsize=(20,10), title='shameless Episodes Watched by Hour')